In [1]:
%load_ext autoreload
%autoreload 2

In [56]:
import sys

import numpy as np
import pandas as pd

import torch
import pickle
import torch.nn as nn
import torch.optim as optim
from torchvision import models

sys.path.append('../')

from data.main import get_dataloaders

In [3]:
with open("../scripts/best_model_wts-1.pkl", "rb") as output_file:
    best_model_wts = pickle.load(output_file)

In [22]:
model = models.vgg16(pretrained=True)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 5)

In [23]:
model.load_state_dict(best_model_wts)

<All keys matched successfully>

In [24]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [36]:
datasets, dataloaders = get_dataloaders(phases=['val'], shuffle=False)

In [37]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

softmax = nn.Softmax(dim=1)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
best_loss = sys.float_info.max

y_pred_proba = []
labels_master = []
        
model.eval()  
running_loss = 0.0
for i, data in enumerate(dataloaders['val']):
    # get the inputs; data is a list of [inputs, labels]
    inputs = data['image']
    labels = data['label']
    inputs = inputs.to(device)
    labels = labels.to(device)

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    model = model.to(device)
    with torch.set_grad_enabled(False):
        outputs = model(inputs)
#         print(outputs)
        pred_proba_batch = softmax(outputs).to("cpu").tolist()
        y_pred_proba += pred_proba_batch
        labels_master += labels.to("cpu").tolist()
        loss = criterion(outputs, labels)

    # print statistics
    running_loss += loss.item() * inputs.size(0)
    if i % 50 == 49:    # print every 50 mini-batches
        print('[%5d] loss: %.3f' %
              (i + 1, running_loss / ((i+1)*inputs.size(0))))

epoch_loss = running_loss / datasets['val'].__len__()
print('Val Loss: {:.4f}'.format(epoch_loss))

[   50] loss: 0.518
[  100] loss: 0.536
[  150] loss: 0.537
Val Loss: 0.5382


In [50]:
y_pred_proba = np.array(y_pred_proba)
y_true = np.array(labels_master)

In [51]:
labels_master

array([0, 0, 0, ..., 0, 0, 0])

In [52]:
y_pred = np.argmax(y_pred_proba, axis=1)

In [72]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score

In [57]:
conf_mat = confusion_matrix(y_true, y_pred)
df_conf_mat = pd.DataFrame(data=conf_mat)

In [60]:
df_conf_mat.columns = pd.MultiIndex.from_product([['Predicted'], df_conf_mat.columns])
df_conf_mat.index = pd.MultiIndex.from_product([['GT'], df_conf_mat.index])

In [61]:
df_conf_mat

Predicted                 
             0   1    2   3   4
GT 0      3759   6  100   1   6
   1       288  14   63   0   1
   2       261   7  462  43  21
   3        10   0   55  56  10
   4         6   0   24  10  66

In [70]:
for i in range(5):
    print(i, precision_score(y_true, y_pred, labels=[i], average='micro'))

0 0.8693339500462535
1 0.5185185185185185
2 0.65625
3 0.509090909090909
4 0.6346153846153846


In [71]:
for i in range(5):
    print(i, recall_score(y_true, y_pred, labels=[i], average='micro'))

0 0.9708161157024794
1 0.03825136612021858
2 0.5818639798488665
3 0.42748091603053434
4 0.6226415094339622


In [75]:
for i in range(5):
    print(i, accuracy_score(y_true, y_pred))

0 0.8269121275384324
1 0.8269121275384324
2 0.8269121275384324
3 0.8269121275384324
4 0.8269121275384324
